In [1]:
import tensorflow as tf
import subprocess
import os
import pandas as pd
import random
import shutil
import numpy as np
import typing

from typing import Any, Tuple
from google.colab import drive #if use colab
from tensorflow.nn import relu, tanh, softmax
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
# if use colab
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#if use colab
git_dir = "/content/IOH-Chat-App"
git_url = "https://github.com/bangkit-team/IOH-chat-app.git"

if not os.path.exists(git_dir):
  subprocess.call(["git", "clone", git_url])

In [4]:
filedir = "/content/IOH-chat-app/Machine Learning/datasets/translation/result/eng-ind.csv" # #if use colab
# filedir = "../..//datasets/translate sentence/result/eng-ind.csv" #if use local env

In [5]:
start_mark = "<start>"
end_mark = "<end>"

In [6]:
class TranslatorDataset:
  def __init__(self, filedir):
    self.filedir = filedir
    self.input_tokenizer = None
    self.target_tokenizer = None
    self._load_data_from_file()

  def _load_data_from_file(self):
    df = pd.read_csv(self.filedir)

    input_lang = df.English.values
    target_lang = df.Indonesia.values

    return input_lang, target_lang

  def _normalize_and_preprocess(self, text):
    text = text.lower().strip()
    text = start_mark + " " + text
    text = text + " " + end_mark

    return text

  def _tokenize(self, sentences, num_words, maxlen):
    punctuation = '!"#$%&()*+,-./:;=?@[\\]^_`{|}~\t\n'

    tokenizer = Tokenizer(num_words=num_words, filters=punctuation)
    tokenizer.fit_on_texts(sentences)

    sequences = tokenizer.texts_to_sequences(sentences)
    sequences = pad_sequences(sequences, maxlen=maxlen, padding="post")

    return sequences, tokenizer

  def _create_dataset(self):
    input_lang, target_lang = self._load_data_from_file()

    input_sentence = np.array(
        list(map(self._normalize_and_preprocess, input_lang)))
    
    target_sentence = np.array(
        list(map(self._normalize_and_preprocess, target_lang)))
    
    return input_sentence, target_sentence

  def _load_dataset(self, num_words):
    input_lang, target_lang = self._create_dataset()

    self.maxlen = max([len(i)for i in input_lang]) // 5
    self.buffer_size = len(input_lang)

    input_sequences, input_tokenizer = self._tokenize(
        input_lang, num_words, self.maxlen)
    
    target_sequences, target_tokenizer = self._tokenize(
        target_lang, num_words, self.maxlen,)

    return (input_sequences, input_tokenizer), (target_sequences, target_tokenizer)
  
  def get(self, num_words, batch_size):
    input, target = self._load_dataset(num_words)

    input_sequences, self.input_tokenizer = input
    target_sequences, self.target_tokenizer = target

    dataset = tf.data.Dataset.from_tensor_slices((input_sequences, target_sequences))
    dataset = dataset.shuffle(self.buffer_size).batch(batch_size, drop_remainder=True)

    return self.input_tokenizer, self.target_tokenizer, dataset

In [7]:
num_words = 6000
batch_size = 64

In [8]:
translator_dataset = TranslatorDataset(filedir)
input_tokenizer, target_tokenizer, dataset = translator_dataset.get(num_words, batch_size)

In [9]:
input_batch, target_batch = next(iter(dataset))

In [10]:
input_batch.shape, target_batch.shape

(TensorShape([64, 35]), TensorShape([64, 35]))

In [11]:
input_vocab_size = len(input_tokenizer.index_word) + 1
target_vocab_size = len(target_tokenizer.index_word) + 1
input_maxlen = input_batch.shape[1]
target_maxlen = target_batch.shape[1]

input_maxlen, target_maxlen, input_vocab_size, target_vocab_size

(35, 35, 4093, 4876)

In [12]:
input_example = input_batch[-1]
input_example

<tf.Tensor: shape=(35,), dtype=int32, numpy=
array([   1,  313,   21, 1390,    2,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0], dtype=int32)>

In [13]:
target_example = target_batch[-1]
target_example

<tf.Tensor: shape=(35,), dtype=int32, numpy=
array([   1,   37, 1913,    2,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0], dtype=int32)>

In [14]:
embed_dims = 128
units = 512

In [15]:
class Encoder():
  def __init__(self, input_vocab_size, embedding_dims, units):
    self.units = units
    self.input_vocab_size = input_vocab_size
    self.embedding_dims = embedding_dims

    self.embedding = layers.Embedding(self.input_vocab_size, self.embedding_dims)
    self.gru_layer = layers.GRU(self.units,
                                    return_sequences=True,
                                    return_state=True,
                                    recurrent_initializer='glorot_uniform')

  def call(self, inputs):
    embedding = self.embedding(inputs)
    encoder = self.gru_layer(embedding, initial_state=None)

    return encoder

In [16]:
class BahdanauAttention(layers.Layer):
  def __init__(self, units):
    super(BahdanauAttention, self).__init__()
    self.w1 = layers.Dense(units, use_bias=True) 
    self.w2 = layers.Dense(units, use_bias=True) 
    self.fd = layers.Dense(1)

  def call(self, query, values):
    query_with_time_axis = tf.expand_dims(query, 1)
    
    score = self.fd(tf.nn.tanh(
        self.w1(query_with_time_axis) + self.w2(values)))

    attention_weights = softmax(score, axis=1)

    context_vector = attention_weights * values
    context_vector = tf.reduce_sum(context_vector, axis=1)

    return context_vector, attention_weights

In [77]:
class Decoder():
  def __init__(self, output_vocab_size, embedding_dims, units):
    self.units = units
    self.output_vocab_size = output_vocab_size
    self.embedding_dims = embedding_dims

    self.embedding = layers.Embedding(self.output_vocab_size, self.embedding_dims)
    self.gru_layer = layers.GRU(self.units,
                                    return_sequences=True,
                                    return_state=True,
                                    recurrent_initializer='glorot_uniform')
    self.attention = BahdanauAttention(self.units)
    self.dense1 = layers.Dense(self.units, activation=tanh)
    self.dense2 = layers.Dense(self.output_vocab_size)

  def call(self, inputs, en_outputs, state):
    embedding = self.embedding(inputs)
    dec_outputs, dec_state = self.gru_layer(embedding, initial_state=state)
    context_vector, attention_weights = self.attention.call(
        query=dec_outputs, values=en_outputs)
    
    context_and_rnn_output = tf.concat([context_vector, dec_outputs], axis=-1)

    attention_vector = self.dense1(context_and_rnn_output)
    outputs = self.dense2(attention_vector)

    return outputs

In [78]:
encoder = Encoder(input_vocab_size, embed_dims, units)
en_outputs, en_states = encoder.call(input_batch)

en_outputs.shape, en_states.shape

(TensorShape([64, 35, 512]), TensorShape([64, 512]))

In [79]:
decoder = Decoder(target_vocab_size, embed_dims, units)
dec_outputs= decoder.call(target_batch, en_outputs, en_states)

dec_outputs.shape

TensorShape([64, 35, 4876])

In [80]:
lr = 1e-3
epochs = 30

optimizer = tf.keras.optimizers.Adam(lr=lr)
loss = tf.keras.losses.SparseCategoricalCrossentropy(
       from_logits=True, reduction='none')

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [81]:
class TranslatorModel():
  def __init__(self, input_vocab_size, 
               target_vocab_size, 
               embed_dims, 
               units):
    self.input_vocab_size = input_vocab_size
    self.target_vocab_size = target_vocab_size
    self.embed_dims = embed_dims
    self.units = units

    self.encoder = Encoder(self.input_vocab_size, self.embed_dims, self.units)
    self.decoder = Decoder(self.target_vocab_size, self.embed_dims, self.units)

  def get_encoder(self):
    return self.encoder

  def get_decoder(self):
    return self.decoder
  
  def build_model(self):
    en_inputs = layers.Input(shape=(None,))
    en_output, en_state = self.encoder.call(en_inputs)

    dec_outputs = self.decoder.call(en_inputs, en_output, en_state)

    model = Model(inputs=[en_inputs], 
                  outputs=[dec_outputs])
    return model

In [82]:
translator_model = TranslatorModel(
    input_vocab_size,
    target_vocab_size,
    embed_dims,
    units,
)
model = translator_model.build_model()

model.compile(
    optimizer=optimizer,
    loss=loss,
    metrics=["accuracy"]
)

In [83]:
checkpoint_path = "/content/drive/MyDrive/translate/checkpoint/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

callback_early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='loss',
    patience=3, 
    verbose=1)

callback_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    monitor='loss', 
    verbose=1, 
    save_freq=10,
    save_weights_only=True, 
    save_best_only=True)

callbacks = [callback_early_stopping,
             callback_checkpoint]

model.compile(
    optimizer=optimizer,
    loss=loss,
    metrics=["accuracy"]
)

model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 embedding_10 (Embedding)       (None, None, 128)    523904      ['input_3[0][0]']                
                                                                                                  
 embedding_11 (Embedding)       (None, None, 128)    624128      ['input_3[0][0]']                
                                                                                                  
 gru_10 (GRU)                   [(None, None, 512),  986112      ['embedding_10[0][0]']           
                                 (None, 512)]                                               

In [84]:
model.fit(dataset,
          epochs=epochs,
          callbacks=callbacks,
          verbose=1)

Epoch 1/30
  9/137 [>.............................] - ETA: 10s - loss: 5.3853 - accuracy: 0.7090
Epoch 1: loss improved from inf to 5.05873, saving model to /content/drive/MyDrive/translate/checkpoint/cp-0001.ckpt
 19/137 [===>..........................] - ETA: 11s - loss: 3.4311 - accuracy: 0.7647
Epoch 1: loss improved from 5.05873 to 3.34557, saving model to /content/drive/MyDrive/translate/checkpoint/cp-0001.ckpt
 29/137 [=====>........................] - ETA: 12s - loss: 2.7510 - accuracy: 0.7828
Epoch 1: loss improved from 3.34557 to 2.70684, saving model to /content/drive/MyDrive/translate/checkpoint/cp-0001.ckpt
 39/137 [=======>......................] - ETA: 11s - loss: 2.3720 - accuracy: 0.7938
Epoch 1: loss improved from 2.70684 to 2.34201, saving model to /content/drive/MyDrive/translate/checkpoint/cp-0001.ckpt
 49/137 [=========>....................] - ETA: 10s - loss: 2.1363 - accuracy: 0.8002
Epoch 1: loss improved from 2.34201 to 2.11923, saving model to /content/drive/

In [85]:
# if use colab
saved_model_path = "/content/drive/MyDrive/translate/saved_model/translate.h5"

# if use local env
# saved_model_path = "code/translate sentence/saved_model/translate.h5"
saved_model_dir = os.path.dirname(saved_model_path)

if os.path.exists(saved_model_dir):
  shutil.rmtree(saved_model_dir)
model.save(saved_model_path)

In [135]:
class Translator(tf.Module):
  def __init__(self, model_path, input_tokenizer, target_tokenizer, maxlen):
    self.input_tokenizer = input_tokenizer
    self.target_tokenizer = target_tokenizer
    self.maxlen = maxlen
    self.model_path = model_path

    self.start_token = input_tokenizer.word_index["<start>"]
    self.end_token = target_tokenizer.word_index["<end>"]

    self._load_model()

  def _load_model(self):
    self.model = tf.keras.models.load_model(self.model_path)

  def _normalize_and_preprocess(self, text):
    punctuation = '!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n'
    
    text = text.lower().strip()
    text = text.replace(punctuation, "")

    return text
      
  def translate(self, sentence):
    normalize_sentence = self._normalize_and_preprocess(sentence)

    sequences = input_tokenizer.texts_to_sequences([normalize_sentence])
    sequences = pad_sequences(sequences, maxlen=self.maxlen, padding="post")

    predictions = self.model.predict(sequences)

    index_prediction = list()

    for i in predictions[0]:
      index_prediction.append(np.argmax(i))
    
    result = target_tokenizer.sequences_to_texts([index_prediction])

    print(result)
    # return {'result': result_text}

In [140]:
translator = Translator(
    saved_model_path,
    input_tokenizer, 
    target_tokenizer,
    input_maxlen,
)

translator.translate("we")

['<start> tolong <end> <end>']
